In [4]:
import codecs
import glob
import math
import os
from decimal import Decimal
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline


plt.rcParams["font.family"] = "Times New Roman"

data_dir = "data/input/励起状態/beta/OH/fc/*.out"
result_file = data_dir.split('励起状態')[1].split('*')[0].replace('/', '_')[1:-1]
result_dir = "data/output/" + result_file + ".csv"
paths = [p for p in glob.glob(data_dir)]

excited_state_1 = "Excited State   1"
excited_state_2 = "Excited State   2"
excited_state_3 = "Excited State   3"

gs_high_model_key = "RB3LYP"
initial_key_1 = "Excited State   1"
end_key_1 = "SavETr"
initial_key_2 = "low   system:  model energy"
end_key_2 = "ONIOM: calculating electric field derivatives."
result_dir = "data/output/result.csv"
df_output = pd.DataFrame()

for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        # initial_key_1を含む行を探す
        initial_key_1_df = df[df['Col_1'].str.contains(initial_key_1, case=False)]
        initial_key_1_index = initial_key_1_df.index[0]
        df_1 = df.iloc[initial_key_1_index:]
        # initial_key_1を含む行以降のend_key_1を含む行を抜き出す
        end_key_1_df = df_1[df_1['Col_1'].str.contains(end_key_1)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_1_index = end_key_1_df.index[0] - initial_key_1_index
        df_1 = df_1.iloc[:end_key_1_index]
        # 該当行の抜き出しが完了
        
        S1_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_1)]['Col_1'].iloc[0].split(' ') if a != '']
        f_1 = float(S1_data[-2].split('f=')[-1])
        E_1 = float(S1_data[4])
        nm_1 = float(S1_data[-4])
        
        S2_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_2)]['Col_1'].iloc[0].split(' ') if a != '']
        f_2 = float(S2_data[-2].split('f=')[-1])
        E_2 = float(S2_data[4])
        nm_2 = float(S2_data[-4])
        
        S3_data = [a for a in df_1[df_1['Col_1'].str.contains(excited_state_3)]['Col_1'].iloc[0].split(' ') if a != '']
        f_3 = float(S3_data[-2].split('f=')[-1])
        E_3 = float(S3_data[4])
        nm_3 = float(S3_data[-4])
    
        data_list = [f_1, E_1, nm_1, f_2, E_2, nm_2, f_3, E_3, nm_3]
        df_parameters = pd.DataFrame(data=data_list, 
                            columns=[torsion_angle],
                            index=["f_1", "S1", "nm_1", "f_2", "S2", "nm_2", "f_3", "S3", "nm_3"]).T
        
        
        
        gs_high_model = float([a for a in df[df['Col_1'].str.contains("RB3LYP")]['Col_1'].iloc[0].split(' ') if a != ''][4])
        initial_key_2_df = df[df['Col_1'].str.contains(initial_key_2, case=False)]
        initial_key_2_index = initial_key_2_df.index[0]
        df_2 = df.iloc[initial_key_2_index:]
        # initial_key_2を含む行以降のend_keyを含む行を抜き出す
        end_key_2_df = df_2[df_2['Col_1'].str.contains(end_key_2)]
        # dfを更新しているのでここでの行番号は0から始まる
        end_key_2_index = end_key_2_df.index[0] - initial_key_2_index
        df_2 = df_2.iloc[:end_key_2_index]
        # 該当行の抜き出しが完了
        
        # 各エネルギー値の抜き出し
        low_model = float(list(df_2['Col_1'])[0].split(":")[-1])
        high_model = float(list(df_2['Col_1'])[1].split(":")[-1])
        low_real = float(list(df_2['Col_1'])[2].split(":")[-1])
        high_real = float(list(df_2['Col_1'])[3].split("=")[-1])
        
        S1_high_real = gs_high_model+(E_1/27.2116) + low_real - low_model
        S2_high_real = gs_high_model+(E_2/27.2116) + low_real - low_model
        S3_high_real = gs_high_model+(E_3/27.2116) + low_real - low_model
        
        df_E = pd.DataFrame(data=[S1_high_real, S2_high_real, S3_high_real, high_real, low_real, high_model, low_model], 
                            columns=[torsion_angle],
                            index=["S1_high_real", "S2_high_real", "S3_high_real", "high real", "low real", "high model", "low model"]).T
        df_result = pd.concat([df_parameters, df_E], axis=1)
        
        df_output = pd.concat([df_output, df_result]).sort_index()

In [5]:
df_output

,f_1,S1,nm_1,f_2,S2,nm_2,f_3,S3,nm_3,S1_high_real,S2_high_real,S3_high_real,high real,low real,high model,low model
-165,0.4852,3.6974,335.33,0.0194,3.9484,314.01,0.2477,4.3343,286.05,-9420.599909,-9420.590685,-9420.576504,-9420.599907,-9416.353878,-631.988898,-627.742869
-150,0.3998,3.7582,329.91,0.0104,3.9690,312.38,0.1315,4.5031,275.33,-9420.599665,-9420.591919,-9420.572291,-9420.599666,-9416.356297,-631.986058,-627.742689
-135,0.2918,3.8665,320.66,0.0077,4.0213,308.32,0.0548,4.6399,267.21,-9420.596319,-9420.590630,-9420.567897,-9420.596318,-9416.357388,-631.979901,-627.740971
-120,0.1621,3.9847,311.15,0.0161,4.0975,302.59,0.0505,4.7834,259.19,-9420.591343,-9420.587198,-9420.561992,-9420.591344,-9416.357370,-631.972486,-627.738512
-60,0.1921,3.7913,327.02,0.1104,4.1109,301.60,0.1463,4.6582,266.16,-9420.613096,-9420.601351,-9420.581238,-9420.613094,-9416.371058,-631.986549,-627.744514
-45,0.2413,3.6569,339.04,0.2089,4.1070,301.89,0.1727,4.5409,273.04,-9420.625787,-9420.609246,-9420.593301,-9420.625785,-9416.377823,-631.998813,-627.750852
-30,0.2799,3.5889,345.47,0.3001,4.1051,302.02,0.1556,4.4636,277.77,-9420.633851,-9420.614881,-9420.601706,-9420.633851,-9416.382727,-632.007164,-627.756039
-15,0.3080,3.5480,349.45,0.3885,4.1110,301.59,0.1053,4.3875,282.59,-9420.638052,-9420.617362,-9420.607201,-9420.638052,-9416.385088,-632.012079,-627.759116
0,0.3259,3.5287,351.35,0.4974,4.1323,300.04,0.0081,4.3197,287.02,-9420.638599,-9420.616418,-9420.609531,-9420.638597,-9416.384813,-632.013696,-627.759912
15,0.3268,3.5471,349.53,0.4179,4.1250,300.57,0.0749,4.3519,284.89,-9420.635098,-9420.613861,-9420.605523,-9420.635096,-9416.382021,-632.011917,-627.758841


In [6]:
df_output.to_csv("data/output/" + result_file + "_all_data.csv")